<a href="https://colab.research.google.com/github/cosimo-schiavoni/Massive_Data_Project/blob/main/Downloads_TFOS_ERROR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install required packages

!wget -q https://www-eu.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
!tar xf spark-3.2.1-bin-hadoop2.7.tgz
!rm spark-3.2.1-bin-hadoop2.7.tgz
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install findspark
!pip install pyspark
!pip install -q kaggle
!pip install tensorflowonspark
!pip install sparkdl
!pip install tensorframes
!pip install petastorm

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 53.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=2db1513201c74ad69be9cd59ad0ca6920bbcb187981bd60685911b4d85c2bc00
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
     |████████████████████████████████| 45 kB 2.6 MB/s 
     |████████████████████████████████| 99 kB 5.8 MB/s 
     |████████████████████████████████| 284 kB 20.3 MB/s 
     |████████████████████████████████| 140 kB 31.8 MB/s 
     |████████████████████████████████| 44 kB 2.7 MB/s 


In [ ]:
#import kaggle.json file with personal credentials for API access

from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi

Saving kaggle.json to kaggle.json


In [ ]:
#Download "Faces/comics" dataset from Kaggle API and organize two folders (faces and comics)

import os
import zipfile

os.environ["data_source"] = "./Face_Comics_data"

api = KaggleApi()
api.authenticate()

if not os.path.exists(os.environ["data_source"]):
  os.makedirs(os.environ["data_source"])
  if "comic-faces-paired-synthetic-v2" not in os.listdir():
    if "comic-faces-paired-synthetic-v2.zip" not in os.listdir():
      ! kaggle datasets download -d defileroff/comic-faces-paired-synthetic-v2
      with zipfile.ZipFile("comic-faces-paired-synthetic-v2.zip", 'r') as f:
        f.extractall("comic-faces-paired-synthetic-v2")
    os.remove("comic-faces-paired-synthetic-v2.zip")


!mv "./comic-faces-paired-synthetic-v2/face2comics_v2.0.0_by_Sxela/face2comics_v2.0.0_by_Sxela/comics" "./Face_Comics_data"
!mv "./comic-faces-paired-synthetic-v2/face2comics_v2.0.0_by_Sxela/face2comics_v2.0.0_by_Sxela/faces" "./Face_Comics_data"

!rm -rf comic-faces-paired-synthetic-v2

100% 2.17G/2.18G [00:22<00:00, 81.2MB/s]
100% 2.18G/2.18G [00:22<00:00, 104MB/s] 


In [ ]:
#Run the Tf.Estimator on Pyspark (TFOS)

#Import some libraries:
#Pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
# Keras / Deep Learning
import tensorflow as tf
from tensorflowonspark import TFParallel
from tensorflowonspark import TFCluster
import argparse
#Other Libraries
import os

# def main function:
def inference():

  if __name__ == '__main__': 

    #Start Spark session and context
    spark = SparkSession.builder.appName("Pyspark on Google Colab")
    sc = SparkContext(conf=SparkConf().setAppName("TFOS"))
    conf = SparkConf().setMaster("local[*]").setAppName("Colab")
    executors = sc._conf.get("spark.executor.instances")
    #Define parameters to parse
    num_executors = int(executors) if executors is not None else 1
    num_ps=1

    #Define Parser
    parser = argparse.ArgumentParser()
    parser.add_argument("--cluster_size", help="number of nodes in the cluster (for Spark Standalone)", type=int, default=num_executors)
    parser.add_argument("--num_ps", help="number of parameter servers", type=int, default=num_ps)
    args = parser.parse_args()

    #Define some parameters for the CNN Keras Sequential model
    data_dir= "./Face_Comics_data"
    tf.random.set_seed(123456)
    BATCH_SIZE = 32
    IMG_SIZE = (350, 350)
    os.listdir(data_dir)

    ###Create CNN Keras Sequential Model
    #Initialize the CNN
    cnn = tf.keras.models.Sequential()
          
    #DROPOUT
    #Introduce the Convolution layer with Kernel_initializer=Sobel_x
    #Convolution
    cnn.add(tf.keras.layers.Conv2D(32, kernel_size=5, strides=2, activation='relu'))
    #Pooling
    cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))
    #Convolution
    cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3,  strides=2, activation = 'relu'))
    #Pooling
    cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))
    #Second Convolutional Layer
    cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 2,  strides=2, activation = 'relu'))
    cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides=1, padding='same'))
    #Flattening
    cnn.add(tf.keras.layers.Flatten())
    #Full Connection
    cnn.add(tf.keras.layers.Dense(units = 128, activation = 'relu'))
    #DROPOUTS ok(0.1)
    cnn.add(tf.keras.layers.Dropout(0.1))
    #Output Layer
    cnn.add(tf.keras.layers.Dense(units=1, activation = 'sigmoid'))
      
    #Compiling the CNN
    cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
        
    #Building the CNN 
    cnn.build(input_shape = [None,350,350,3]) # `input_shape` is the shape of the input data
    cnn.summary()

    #Convert the CNN model in a tf.estimator.train_and_evaluate
    from keras.preprocessing.image import ImageDataGenerator
    # create generator object
    datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2)

    #define train test input function
    @tf.function     
    def train_input_fn(): 
      train_it = datagen.flow_from_directory(
      data_dir, 
      class_mode='binary',
      subset='training', 
      color_mode="rgb", 
      shuffle=True,
      seed=123456,
      batch_size=BATCH_SIZE, 
      target_size=IMG_SIZE
      )
      features, labels = train_it.next()
      print('Batch shape=%s, min=%.3f, max=%.3f' % (features.shape, features.min(), features.max()))
      return features, labels
        
    #define validation test input function
    @tf.function 
    def eval_input_fn():
      val_it = datagen.flow_from_directory(
      data_dir, 
      class_mode='binary',
      subset='validation', 
      color_mode="rgb", 
      shuffle=True,
      seed=123456,
      batch_size=BATCH_SIZE, 
      target_size=IMG_SIZE
      )
      features, labels = val_it.next()
      print('Batch shape=%s, min=%.3f, max=%.3f' % (features.shape, features.min(), features.max()))
      return features, labels

    #define the estimator
    import tempfile
    model_dir = tempfile.mkdtemp()
    keras_estimator = tf.keras.estimator.model_to_estimator(
      keras_model=cnn, model_dir=model_dir)

    #Train and evaluate
    train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=1000)
    eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn)

    tf.estimator.train_and_evaluate(keras_estimator, train_spec, eval_spec)
  #define parallel run of estimator in Spark environment

      #TFCluster.run(sc, main_fun, args, args.cluster_size, args.num_ps, TFCluster.InputMode.TENSORFLOW)
    TFParallel.run(sc, inference, args, args.cluster_size, use_barrier=False)
    
inference()

usage: ipykernel_launcher.py [-h] [--cluster_size CLUSTER_SIZE]
                             [--num_ps NUM_PS] [--tensorboard]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-7aa3f316-ee26-49e8-9d72-7814d9a48255.json


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
